# Google Colab Connect

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import statements 

In [2]:
from sklearn.cluster import KMeans
import numpy as np
import cv2
import math
import scipy.spatial.distance as d
from scipy.spatial import distance

from PIL import Image


In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure 
import json
import datetime
import copy
from PIL import Image as im 
import joblib

In [4]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [5]:
import scipy.cluster.vq as vq
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt

# Saving and Loading function 

In [6]:
# Saving and Loading models using joblib 
def save(filename, obj):
  with open(filename, 'wb') as handle:
      joblib.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load(filename):
  with open(filename, 'rb') as handle:
      return joblib.load(filename)

# Arguments and data path 

In [7]:
class Args():
  def __init__(self,data_path,dataset):
    self.train_f = os.path.join(data_path,'all_img_features.pkl')
    self.train_name =  os.path.join(data_path,'all_img_names.pkl')
    self.img_dict = os.path.join(data_path,'data.pkl')
    self.Hist_all = os.path.join(data_path,'Hist_all')
    self.train_amap = os.path.join(data_path,'assignment_map_250.pkl')
    self.k_mean_centers = os.path.join(data_path,'k_mean_centers.pkl')
    self.ground_truth = os.path.join(data_path,dataset +'_GT_final')
    self.ranking_one_path = os.path.join(data_path)
    self.datasetName = dataset
    self.all_original_images = '/content/drive/MyDrive/SUB/PROJECTS/IR_final/datasets/all_images/'+ dataset +'/'

# Datasets and Loading required files 

In [8]:
# my_paths = Args('/content/drive/MyDrive/SUB/PROJECTS/IR_final/models/baseline_dl_model /paris/others','paris')
my_paths = Args('/content/drive/MyDrive/SUB/PROJECTS/IR_final/models/baseline_dl_model /oxford/others','oxford')
# my_paths = Args('/content/drive/MyDrive/SUB/PROJECTS/IR_final/models/baseline_dl_model /instre/others','instre')
# my_paths = Args('/content/drive/MyDrive/SUB/PROJECTS/IR_final/models/baseline_dl_model /sculpture/others','sculpture')

In [9]:
train_f = load(my_paths.train_f)
train_name = load(my_paths.train_name)
img_dict = load(my_paths.img_dict)
Hist_all = load(my_paths.Hist_all)
train_amap = load(my_paths.train_amap)
k_mean_centers = load(my_paths.k_mean_centers)
ground_truth = load(my_paths.ground_truth)

In [10]:
original_data_path = my_paths.all_original_images 
original_data_path

'/content/drive/MyDrive/SUB/PROJECTS/IR_final/datasets/all_images/oxford/'

## Useful functions

In [11]:
def input_vector_encoder(feature, codebook):
    """
    Input all the local feature of the image
    Pooling (encoding) by codebook and return
    """
    code, _ = vq.vq(feature, codebook)
    word_hist, bin_edges = np.histogram(code, bins=range(codebook.shape[0] + 1))
    return word_hist

### Weighted cosine

In [12]:
def apply_weighted_cosineSimilarity(images_bow,query_bow, th = 0.8):
  weights = np.ones(query_bow.shape[1])
  for idx,ele in enumerate(query_bow[0]):
    if ele == 0:
      weights[idx] = th 

  l2_norm_db = np.sum(weights * np.abs(images_bow)**2,axis=-1)**(1./2)
  l2_norm_db =l2_norm_db.reshape((l2_norm_db.shape[0],1))
  l2_norm_query = np.sum(weights * np.abs(query_bow)**2,axis=-1)**(1./2)
 
  l2_norm_query =l2_norm_query.reshape((l2_norm_query.shape[0],1))
  
  l2_norm = l2_norm_db * l2_norm_query
  
  mul = query_bow[0]* weights
  mul = np.reshape(mul, (1,mul.shape[0]))
  dot_product = images_bow @ (mul).T
  
  
  cosine_similarity = dot_product/l2_norm
  
  return cosine_similarity

def compute_similarity(images_bow,query_bow,dist_type = 'cosine'):
  if dist_type == 'weighted_cosine':
    return apply_weighted_cosineSimilarity(images_bow,query_bow, th = 0.8)
  elif dist_type == 'cosine':
    return cosineSimilarity(images_bow,query_bow)

In [13]:
def cosineSimilarity(images_bow,query_bow):
  
  l2_norm_db = np.sum(np.abs(images_bow)**2,axis=-1)**(1./2)
  l2_norm_db =l2_norm_db.reshape((l2_norm_db.shape[0],1))
  l2_norm_query = np.sum(np.abs(query_bow)**2,axis=-1)**(1./2)
 
  l2_norm_query =l2_norm_query.reshape((l2_norm_query.shape[0],1))
  
  l2_norm = l2_norm_db * l2_norm_query
  dot_product = images_bow @ query_bow.T
  
  cosine_similarity = dot_product/l2_norm
  
  return cosine_similarity

def topNmatches(images_bow, query_bow,N,dist_type):
  cosine_similarity = compute_similarity(images_bow,query_bow,dist_type)
  highest_matching_image_indices = np.argsort(cosine_similarity,axis = 0)[::-1][:cosine_similarity.shape[0]]
  highest_matching_image_indices = highest_matching_image_indices.reshape((highest_matching_image_indices.shape[0],))
  if N is None:
    return highest_matching_image_indices[0:N]
  else: 
    return highest_matching_image_indices[0:N]

In [14]:
def compute_cosine(a,b):
  return 1 - d.cosine(a, b)

def get_hist_amap(code, codebook):
    """
    Input all the local feature of the image
    Pooling (encoding) by codebook and return
    """

    word_hist, bin_edges = np.histogram(code, bins=range(codebook.shape[0] + 1))
    return word_hist

    #for finding bounding box in assignment_map

def get_map_patch(image_filename,image_width,image_height,assignment_map,x1,y1,x2,y2):
  bb_width =  assignment_map.shape[0]
  bb_height = assignment_map.shape[1]
  # print(bb_width,bb_height)
  width_ratio = bb_width/image_width
  height_ratio = bb_height/image_height
  # print(width_ratio,height_ratio)
  a_map_x1 = int(x1*width_ratio)
  a_map_x2 = int(x2*width_ratio)

  a_map_y1 = int(y1*width_ratio)
  a_map_y2 = int(y2*width_ratio)
  # print(a_map_x1,a_map_x2,a_map_y1,a_map_y2)
  return assignment_map[a_map_x1:a_map_x2 + 1,a_map_y1:a_map_y2 + 1]


def get_assignment_map(feature, codebook):
    """
    Input all the local feature of the image
    Pooling (encoding) by codebook and return
    """
    code, _ = vq.vq(feature, codebook)
    return code

In [15]:
# given top images map and bounding box histogram generated reranked images N images 
def re_rank_scores(bb_hist, top_images_amap, k_mean_centers,top_images_idx, N):
  scores = []
  coordinates = []
  my_x1 , my_y1, my_x2, my_y2 = 0,0,0,0

  # # getting height and width of original image
  # img = cv2.imread(original_data_path+ test_img_jpg)
  # height, width, channels = img.shape

  for idx,curr_amap in enumerate(top_images_amap):
    max = 0 
    # window size x1, y1
    # Image assignment height and width W, H 

    all_wid = [(2,2),(4,4),(6,6)]
    for (x1,y1) in all_wid:
      # x1,y1 = 4,4
      H,W = 7,7
      # stride in x and y
      s_x = 1 
      s_y = 1
      y_moves = math.floor(((H-y1)/s_y)) + 1
      x_moves = math.floor(((W-x1)/s_x)) + 1
      y_dx = 0
      x_dx = 0

      for y in range(y_moves):
        x_dx = 0
        for x in range(x_moves):
            # print(y_dx,y_dx+y1)
            # print(x_dx,x_dx+x1)

            # print(curr_amap)
            window_amap  = curr_amap[y_dx:y_dx+y1,x_dx:x_dx+x1]
            window_hist = get_hist_amap(window_amap, k_mean_centers)
            # print(window_hist)
            if max < compute_cosine(bb_hist,window_hist):
              max = compute_cosine(bb_hist,window_hist)
              
              # getting coordinates in current window 
              my_x1 , my_y1, my_x2, my_y2 = x_dx,y_dx,x_dx+x1,y_dx+y1

            # print(input_vector_encoder1(temp,k_mean_centers))
            x_dx =x_dx+s_x
        y_dx = y_dx + s_y

    scores.append(max)
    # rescale these later according to display image size 
    coordinates.append([my_x1/7,my_y1/7,my_x2/7,my_y2/7])


  zipped_lists = zip(scores[:N], top_images_idx[:N])
  sorted_pairs = sorted(zipped_lists)

  zipped_lists2 = zip(scores[:N], coordinates[:N])
  sorted_pairs2 = sorted(zipped_lists2)

  tuples = zip(*sorted_pairs)
  list1, list2 = [ list(t) for t in tuples]

  tuples2 = zip(*sorted_pairs2)
  list12, list22 = [ list(t) for t in tuples2]

  list1.reverse() , list2.reverse()
  list12.reverse() , list22.reverse()
  # returns reversed image indices 

  final_names = np.take(train_name, list2)
  # print(final_names)
  # coordinates values sent in list22
  return final_names , list22


In [16]:
# return index of train images which are top match 

def find_top_matches(query_bow , retrive_N , Hist_all, train_name ):
  query = query_bow.reshape(1,-1)
  top_images = topNmatches(np.array(Hist_all),query, retrive_N )
  retrieved_names = np.take(train_name, top_images)
  return top_images , retrieved_names

def display_images(img_name):
  im_rgb = cv2.cvtColor(np.array(img_dict[img_name]), cv2.COLOR_BGR2RGB)
  plt.imshow(im_rgb)
  plt.show()

# def convert(bgr):
#   return cv2.cvtColor(np.array(img_dict[img_name]), cv2.COLOR_BGR2RGB)


# given index of top images return amap of only top images for second search 
def get_amap_of_top_images(top_images_idx , train_amap):
  return np.take(train_amap ,top_images_idx, 0)

def get_query_amap(query_name, train_amap, train_name):
  idx = train_name.index(query_name)
  amap = train_amap[idx]
  # print(amap.shape)
  return amap 

# Image rank for one image 

In [17]:
# returns names of images top N which matches to current image
def get_top_images_ranking_one(image_name, train_name, Hist_all ,dist_type, N_retrive = None):
  # fetch all things 
  image_id = train_name.index(image_name)
  images_bow = np.array(Hist_all)
  query_hist  = images_bow[image_id]
  query_hist = query_hist.reshape(1,-1)

  # perform ranking one and matching 
  top_images_idx = topNmatches(images_bow,query_hist, N_retrive,dist_type)
  
  retrieved_names = np.take(train_name,top_images_idx)
  return top_images_idx , retrieved_names

In [18]:
def get_top_images_ranking_two(image_name ,train_name, Hist_all ,k_mean_centers,N_retrive, top_images_idx , retrieved_names, train_amap,x1,y1,x2,y2,h_img=224, w_img=224):

  # get amap of train images top 
  top_images_amap = get_amap_of_top_images(top_images_idx , train_amap)

  # get amap of query 
  query_amap = get_query_amap(image_name, train_amap, train_name)

  # generating histogram for bounding box 
  bb_amap = get_map_patch(image_name,h_img,w_img,query_amap,x1,y1,x2,y2)
  bb_hist = get_hist_amap(bb_amap, k_mean_centers)

  # comparing to half windows gives the best results 
  final_names, coordinates_final_names = re_rank_scores(bb_hist, top_images_amap, k_mean_centers,top_images_idx, N_retrive)
  return final_names, coordinates_final_names


# Evaluation 

In [19]:
# def find_MAP(labelled_arr):
#   count = 0
#   rel = 0
#   true_pre = 0

#   for label in labelled_arr:
#     count = count + 1
#     if label == 1:
#       rel = rel + 1 
#       true_pre = true_pre + rel/count 
#       # print("true Precision for relevant query for label", rel/count )
#   #print("count", count)  
#   if rel == 0:
#     return 0
#   else:
#     avgPre = true_pre/(rel)
#     #print("rel", rel)
#     return avgPre

## Mean Average Precision 

In [20]:
# labelled array is an array of 0,1,0,1  and rev_idx gives number of relevant docs to be considered for mean avg precision
def find_MAP(labelled_arr, rev_idx):
  count = 0
  rel = 0
  true_pre = 0
  rel_enc = 0
  for label in labelled_arr:
    count = count + 1
    if rel_enc < rev_idx:
      if label == 1:
        rel_enc = rel_enc + 1
        rel = rel + 1 
        true_pre = true_pre + rel/count 
      # print("true Precision for relevant query for label", rel/count )
  #print("count", count)  
  if rel == 0:
    return 0
  else:
    avgPre = true_pre/(rel)
    
    return avgPre

## Get Relevancy map for each image according to ground truth 

In [21]:
# original list contains extension .jpg


def get_map(query_image_name, top_result_names):
  def get_good_ok_images(query_image_name):
    # print(query_image_name)
    if '.' in query_image_name:
      query_image_name = query_image_name[:query_image_name.index('.')]
      # print(query_image_name)
    if query_image_name in ground_truth.keys():
      obj = ground_truth.get(query_image_name, dict())
      # print(obj)
      goodNames = []
      if 'good' in obj.keys():
        goodNames = list(obj.get('good', []))
      else:
        # print('no good names...')
        pass
      if 'ok' in obj.keys():
        goodNames.extend(obj.get('ok', []))
      else:
        # print('no ok names...')
        pass
      if 'junk' in obj.keys():
        # goodNames.extend(obj.get('junk', []))
        pass
      else:
        # print('no ok names...')
        pass
      if 'ignore' in obj.keys():
        # goodNames.extend(obj.get('junk', []))
        pass
      else:
        # print('no ok names...')
        pass
        
      goodNames = list(set(goodNames))
      # print(goodNames)
      return goodNames
    else:
      # print('query image not present...')
      pass
      return None

  names = get_good_ok_images(query_image_name)
  scores = [0]*len(top_result_names)

  if names is None:
    return scores

  for idx, i in enumerate(top_result_names):
    if i in names:
      scores[idx] = 1
  return scores

## Get Relevancy map for each image according to ground truth [ DGC ]

In [22]:
# original list contains extension .jpg


def DGC_get_map(query_image_name, top_result_names):
  def get_good_ok_images(query_image_name):
    # print(query_image_name)
    if '.' in query_image_name:
      query_image_name = query_image_name[:query_image_name.index('.')]
      # print(query_image_name)
    if query_image_name in ground_truth.keys():
      obj = ground_truth.get(query_image_name, dict())
      # print(obj)
      goodNames = []
      okNames = []
      junkNames = []
      ignoreNames = []

      if 'good' in obj.keys():
        goodNames = list(obj.get('good', []))
      else:
        # print('no good names...')
        pass
      if 'ok' in obj.keys():
        okNames = list(obj.get('ok', []))
      else:
        # print('no ok names...')
        pass
      if 'junk' in obj.keys():
        junkNames = list(obj.get('junk', []))
        pass
      else:
        # print('no ok names...')
        pass
      if 'ignore' in obj.keys():
        ignoreNames = list(obj.get('ignore', []))
        pass
      else:
        # print('no ok names...')
        pass
        
      goodNames = list(set(goodNames))
      # print(goodNames)
      return goodNames, okNames, junkNames, ignoreNames
    else:
      # print('query image not present...')
      pass
      return None

  goodNames, okNames, junkNames, ignoreNames = get_good_ok_images(query_image_name)
  scores = [0]*len(top_result_names)

  if goodNames is None and okNames is None and junkNames is None and ignoreNames is None:
    return scores

  for idx, i in enumerate(top_result_names):
    if i in goodNames:
      scores[idx] = 3
    elif i in okNames:
      scores[idx] = 2
    elif i in junkNames:
      scores[idx] = 1
    elif i in ignoreNames:
      scores[idx] = 1
  return scores

## DGC computation functions 

In [23]:
def compute_dcg_at_k(r, k, m='f1'):
    # get k values 
    r = np.asfarray(r)[:k]
    if r.size:
        if m == 'f1':
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif m == 'f2':
            return np.sum((np.power(2, r) - 1) / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('not valid.')
    return 0

def compute_ndcg_at_k(r, k, m='f1'):
    dcg_max = compute_dcg_at_k(sorted(r, reverse=True), k, m)
    if not dcg_max:
        return 0.
    return compute_dcg_at_k(r, k, m) / dcg_max  

def compute_idcg_at_k(r, k, m='f1'):
    dcg_max = compute_dcg_at_k(sorted(r, reverse=True), k, m)
    if not dcg_max:
        return 0.
    else:
      return dcg_max
# it computes the max dcg value given relevance list and which formula to use 
def compute_idcg(relevance, alternate='f1'):

    if relevance is None or len(relevance) < 1:
        return 0.0
    r = np.asarray(relevance).copy()
    r.sort()
    return dcg(r[::-1], alternate) 

# required functions 

# it computes the dcg value given relevance list and which formula to use 
# f1 -> formula of DCG
# f2 -> alternative formula of DCG 
def compute_dcg(relevance, alternate='f1'):
    # a list of all the relevance scores provided 
    if relevance is None or len(relevance) < 1:
        return 0.0
    rel = np.asarray(relevance)
    p = len(rel)

    if alternate == 'f2':
        #An alternative formulation DCG
        log2i = np.log2(np.asarray(range(1, p + 1)) + 1)
        return ((np.power(2, rel) - 1) / log2i).sum()
    else:
        log2i = np.log2(range(2, p + 1))
        return rel[0] + (rel[1:] / log2i).sum()

# print("DGC", compute_dcg_at_k(relData, 50, m='f1'))
# print("iDGC", compute_idcg_at_k(relData, 50, m='f1'))
# print("nDGC", compute_ndcg_at_k(relData, 50, m='f1') )


# Resize Image and Bounding box function 

In [24]:
from PIL import Image
# dist_type = 'weighted_cosine'
# dist_type = 'weighted_cosine'
dist_type = 'weighted_cosine'

def get_bbx_image(img,p1,p2):
 # img is numpy array of image in BGR format
#  im_arr_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
#  cv2.rectangle(im_arr_bgr, (10,10), (100,100),color=(0, 0, 255), thickness=2)
 cv2.rectangle(img, p1, p2,color=(0, 0, 255), thickness=10)
 img = cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
 im_arr = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
 
 im = Image.fromarray(im_arr)
 # return a PIL image of RGB format 
 return im 

In [25]:

def resize_file(in_file,size):
    image = resizeimage.resize_thumbnail(in_file, size)
    return image
    

In [26]:
# train_name --> all images  idx 0,1,2 , names 
# top_ranking_.pkl  

# Final Results 

In [27]:
# dist_type = 'cosine'
dist_type = 'weighted_cosine'

In [28]:
# ranking all images 

test_image_names = list(ground_truth.keys())
map_test_scores = []
ndgc_scores = []
for idx,test_img in enumerate(test_image_names):
  test_img_jpg = test_img + '.jpg'
  # getting height and width of original image
  img = cv2.imread(original_data_path+ test_img_jpg)

  height, width, channels = img.shape  
  
  x1,y1,x2,y2 = ground_truth[test_img]['annotation']
  x1,y1,x2,y2 = float(x1), float(y1), float(x2), float(y2)

  # x1,y1,x2,y2 = (int(x1), int(y1)), (int(x2), int(y2))
  # plt.imshow(get_bbx_image(img,(x1,y1),(x2,y2)))
  # break 

  top_images_idx , retrieved_names = get_top_images_ranking_one(test_img_jpg, train_name, Hist_all ,dist_type, 100)
  # print(top_images_idx , retrieved_names)
  final_names, coordinates_final_names = get_top_images_ranking_two(test_img_jpg ,train_name, Hist_all ,k_mean_centers,100, top_images_idx , retrieved_names, train_amap,x1,y1,x2,y2,height,width)
  
  plot_names = []
  plot_names_cor = []

  plot_names.append(test_img_jpg)
  plot_names_cor.append([x1/width,y1/height,x2/width,y2/height])
  
  plot_names.extend(final_names)
  plot_names_cor.extend(coordinates_final_names)

  # print(plot_names)
  # print(plot_names_cor)

  filter_train_img_names = [i[:i.index('.')] for i in final_names]
  
  # MAP computation for one image
  eval_arr = get_map(test_img, filter_train_img_names)
  c = find_MAP(eval_arr,20)
  map_test_scores.append(c)
  # nDGC computation for one image 
  eval_arr2 = DGC_get_map(test_img, filter_train_img_names)
  # print(eval_arr2)
  c2 = compute_ndcg_at_k(eval_arr2, len(eval_arr2), m='f1')
  # print(c2)
  ndgc_scores.append(c2)

  fig, axs = plt.subplots(1,6, figsize=(15, 6), facecolor='w', edgecolor='k')
  fig.subplots_adjust(hspace = .5, wspace=.001)
  for i, ax in enumerate(axs.ravel()):

  # for each image in plot names read original image, height , width
    img = cv2.imread(original_data_path+ plot_names[i])
    H, W, Chan = img.shape   
    # coordinates rescaling according to orifinal image 
    p1 = (int(plot_names_cor[i][0]*W),int(plot_names_cor[i][1]*H))
    p2 = (int(plot_names_cor[i][2]*W),int(plot_names_cor[i][3]*H))

    # get bounding box image 
    img = get_bbx_image(img,p1,p2)
    
    # img = cv2.cvtColor(np.array(img_dict[plot_names[i]]), cv2.COLOR_BGR2RGB)
    plt.setp(ax.get_xticklabels(), visible=False)
    plt.setp(ax.get_yticklabels(), visible=False)
    ax.imshow(img)

  if idx%20 == 0:
    print("Images Done till now : ", idx+1 , "Out of : ",len(test_image_names))




Output hidden; open in https://colab.research.google.com to view.

In [29]:
print("DISTANCE METRIC :" ,dist_type)
print("FOR COMPLETE DATASET : ", my_paths.datasetName)
print("MEAN AVERAGE PRECISION VALUE : ", np.array(map_test_scores).mean())

print("MEAN NORMALIZE DICOUNTED CUMMULATIVE GAIN ", np.array(ndgc_scores).mean())


DISTANCE METRIC : weighted_cosine
FOR COMPLETE DATASET :  oxford
MEAN AVERAGE PRECISION VALUE :  0.6961010099367417
MEAN NORMALIZE DICOUNTED CUMMULATIVE GAIN  0.7535930372060692


# DEMO 

## Obtain ranking for single image 

In [30]:
import cv2 
original_data_path = original_data_path
from matplotlib import pyplot as plt

# ranking per image
def get_result(test_img_name, x1=None,y1=None,x2=None,y2=None):
  map_test_score, ndgc_score = 0, 0
  test_img_jpg = test_img_name + '.jpg'
  # getting height and width of original image
  img = cv2.imread(original_data_path+ test_img_jpg)

  height, width, channels = img.shape  
  if x1 is None:
    x1,y1,x2,y2 = ground_truth[test_img_name]['annotation']
    x1,y1,x2,y2 = float(x1), float(y1), float(x2), float(y2)
 

  # x1,y1,x2,y2 = (int(x1), int(y1)), (int(x2), int(y2))
  # plt.imshow(get_bbx_image(img,(x1,y1),(x2,y2)))
  # break

  top_images_idx , retrieved_names = get_top_images_ranking_one(test_img_jpg, train_name, Hist_all ,dist_type, 100)
  # print(top_images_idx , retrieved_names)
  final_names, coordinates_final_names = get_top_images_ranking_two(test_img_jpg ,train_name, Hist_all ,k_mean_centers,100, top_images_idx , retrieved_names, train_amap,x1,y1,x2,y2,height,width)
 
  plot_names = []
  plot_names_cor = []
  images = []

  plot_names.append(test_img_jpg)
  plot_names_cor.append([x1/width,y1/height,x2/width,y2/height])

  plot_names.extend(final_names)
  plot_names_cor.extend(coordinates_final_names)

  # print(plot_names)
  # print(plot_names_cor)

  filter_train_img_names = [i[:i.index('.')] for i in final_names]
 
  # MAP computation for one image
  map_test_score = find_MAP(get_map(test_img_name, filter_train_img_names),20)
  # nDGC computation for one image
  eval_arr2 = DGC_get_map(test_img_name, filter_train_img_names)
  ndgc_score = compute_ndcg_at_k(eval_arr2, len(eval_arr2), m='f1')

  fig, axs = plt.subplots(1,6, figsize=(15, 6), facecolor='w', edgecolor='k')
  fig.subplots_adjust(hspace = .5, wspace=.001)
  
  for i, ax in enumerate(axs.ravel()):
  # for each image in plot names read original image, height , width
    img = cv2.imread(original_data_path+ plot_names[i])
    H, W, Chan = img.shape  
    # coordinates rescaling according to orifinal image
    p1 = (int(plot_names_cor[i][0]*W),int(plot_names_cor[i][1]*H))
    p2 = (int(plot_names_cor[i][2]*W),int(plot_names_cor[i][3]*H))

    # get bounding box image
    img = get_bbx_image(img,p1,p2)
   
    # img = cv2.cvtColor(np.array(img_dict[plot_names[i]]), cv2.COLOR_BGR2RGB)
    plt.setp(ax.get_xticklabels(), visible=False)
    plt.setp(ax.get_yticklabels(), visible=False)
    ax.imshow(img)
 
  print("PRECISION VALUE : ", map_test_score)

  print("NORMALIZE DICOUNTED CUMMULATIVE GAIN ", ndgc_score)



## Display all ground truth images with bounding box 

In [34]:
def Display_all_ground_truth_images(ground_truth):
  names_list = []
  names_list_cord = []
  #  contains all raw names 
  options = []
  counter = -1

  def resize_img(temp,img):
    # img is already read here and temp contains original coordinates of boxes 
    H, W, Chan = img.shape 
    (x1,y1,x2,y2)  = temp
    # coordinates rescaling according to orifinal image 
    p1 = (int(x1),int(y1))
    p2 = (int(x2),int(y2))
    # print(p1,p2)
    # get bounding box image 
    return get_bbx_image(img,p1,p2)
    # return cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)


  for idx, x in enumerate(ground_truth.keys()):

    x1,y1,x2,y2 = ground_truth[x]['annotation']
    x1,y1,x2,y2 = float(x1), float(y1), float(x2), float(y2)

    img = cv2.imread(original_data_path+ x+'.jpg')
    names_list.append(img)
    names_list_cord.append((x1,y1,x2,y2))
    options.append(x)

    if (idx+1)%5==0:
      
      fig, (ax1, ax2,ax3,ax4,ax5) = plt.subplots(1, 5, figsize=(10, 8))
      counter += 1
      if counter*5 + 0 < len(names_list):
        ax1.imshow(resize_img(names_list_cord[counter*5 + 0],names_list[counter*5 + 0]))
      if counter*5 + 1 < len(names_list):
        ax2.imshow(resize_img(names_list_cord[counter*5 + 1],names_list[counter*5 + 1]))
      if counter*5 + 2 < len(names_list):
        ax3.imshow(resize_img(names_list_cord[counter*5 + 2],names_list[counter*5 + 2])) 
      if counter*5 + 3 < len(names_list):
        ax4.imshow(resize_img(names_list_cord[counter*5 + 3],names_list[counter*5 + 3]))
      if counter*5 + 4 < len(names_list):
        ax5.imshow(resize_img(names_list_cord[counter*5 + 4],names_list[counter*5 + 4]))
      
      ax1.xaxis.set_visible(False)
      ax1.yaxis.set_visible(False)

      ax2.xaxis.set_visible(False)
      ax2.yaxis.set_visible(False)

      ax3.xaxis.set_visible(False)
      ax3.yaxis.set_visible(False)

      ax4.xaxis.set_visible(False)
      ax4.yaxis.set_visible(False)

      ax5.xaxis.set_visible(False)
      ax5.yaxis.set_visible(False)

      ax1.title.set_text("Index : " + str(counter*5 + 0+1))
      ax2.title.set_text("Index : "+ str(counter*5 + 1+1))
      ax3.title.set_text("Index : "+ str(counter*5 + 2+1))
      ax4.title.set_text("Index : "+ str(counter*5 + 3+1))
      ax5.title.set_text("Index : " + str(counter*5 + 4+1))

  if len(list(ground_truth.keys()))%5!=0:
    fig, (ax1, ax2,ax3,ax4) = plt.subplots(1, 4, figsize=(10, 8))
    counter = (len(list(ground_truth.keys()))//5)

    if counter*5 + 0 < len(names_list):
      ax1.imshow(resize_img(names_list_cord[counter*5 + 0],names_list[counter*5 + 0]))
      ax1.xaxis.set_visible(False)
      ax1.yaxis.set_visible(False)
      ax1.title.set_text("Index : " + str(counter*5 + 0+1))
    else:
      ax1.axis('off')

    if counter*5 + 1 < len(names_list):
      ax2.imshow(resize_img(names_list_cord[counter*5 + 1],names_list[counter*5 + 1]))
      ax2.xaxis.set_visible(False)
      ax2.yaxis.set_visible(False)
      ax2.title.set_text("Index : "+ str(counter*5 + 1+1))
    else:
      ax2.axis('off')

    if counter*5 + 2 < len(names_list):
      ax3.imshow(resize_img(names_list_cord[counter*5 + 2],names_list[counter*5 + 2])) 
      ax3.title.set_text("Index : "+ str(counter*5 + 2+1))
      ax3.xaxis.set_visible(False)
      ax3.yaxis.set_visible(False)
    else:
      ax3.axis('off')

    if counter*5 + 3 < len(names_list):
      ax4.imshow(resize_img(names_list_cord[counter*5 + 3],names_list[counter*5 + 3]))
      ax4.title.set_text("Index : "+ str(counter*5 + 3+1))
      ax4.xaxis.set_visible(False)
      ax4.yaxis.set_visible(False)
    else:
      ax4.axis('off')
  return options, names_list_cord 


## Display single Image 

In [35]:
def Display_one_image(img,x1=None,y1=None,x2=None,y2=None):
  # img is after imread 
  if x1 is None:
    plt.title('IMAGE')
    plt.imshow(cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_CUBIC))
  else:
    H, W, Chan = img.shape 
    # coordinates rescaling according to orifinal image 
    p1 = (int(x1),int(y1))
    p2 = (int(x2),int(y2))
    plt.title('SELECTED IMAGE WITH BOX')
    plt.imshow(get_bbx_image(img,p1,p2))
    ax = plt.gca()
    ax.axes.xaxis.set_ticks([])
    ax.axes.yaxis.set_ticks([])



## Final Function for demo 

In [36]:
def get_results():

  #  Displaying all   
  options,names_list_cord = Display_all_ground_truth_images(ground_truth)
  plt.pause(0.001)
  x = int(input('Enter a choice: '))
  selected_img_array = cv2.imread(original_data_path+ options[x-1]+'.jpg')
  
  height, width, channels =  cv2.imread(original_data_path+ options[x-1]+'.jpg').shape  
  choice = input(f'Provide annotations [x1, y1, x2, y2] between (0,0) - {width, height} (Y/N): ?\n').lower()
  if choice == 'y':
    x1, y1, x2, y2 = list(map(float, input().split()))
    
    Display_one_image(selected_img_array,x1,y1,x2,y2)

  else:
    x1, y1, x2, y2 = None, None, None, None
    
    x_1,y_1,x_2,y_2  = names_list_cord[x-1][0],names_list_cord[x-1][1],names_list_cord[x-1][2],names_list_cord[x-1][3]
    Display_one_image(selected_img_array,x_1,y_1,x_2,y_2)
     
  get_result(options[x-1], x1, y1, x2, y2)

# TAKE DEMO

In [37]:
import warnings 
warnings.filterwarnings("ignore")

plt.style.use("classic")
plt.rcParams.update({'font.size': 10})
get_results()

Output hidden; open in https://colab.research.google.com to view.